### 13.1 자료의 사전정리

In [1]:
import pandas as pd
import numpy as np
import string

lines = pd.read_table("C:/Users/jongcheol/OneDrive/바탕 화면/Semester2/Statistical-ML/Project4_Last/Study/제13장/english to french/english to french.txt")

In [2]:
lines.shape

(149860, 2)

In [3]:
lines = lines[0:50000]
print(lines.head(5))
print(lines.tail(5))

     Go.        Va !
0   Run!     Cours !
1   Run!    Courez !
2   Wow!  Ça alors !
3  Fire!    Au feu !
4  Help!  À l'aide !
                            Go.                               Va !
49995  They gave us their word.  Elles nous donnèrent leur parole.
49996  They go to work on foot.        Ils vont au travail à pied.
49997  They got into the train.       Ils montèrent dans le train.
49998  They got into the train.     Elles montèrent dans le train.
49999  They got into the train.     Ils sont montés dans le train.


In [4]:
lines.columns = ['eng', 'fr']
lines.columns

Index(['eng', 'fr'], dtype='object')

In [5]:
lines['eng'] = lines['eng'].apply(lambda x: x.lower())
lines['fr'] = lines['fr'].apply(lambda x: x.lower())

exclude = set(string.punctuation)
    # 구두점들을 중복을 허용하지 않는 set 자료형으로 저장

In [6]:
lines.tail(1).eng.apply(lambda x: [print(ch) for ch in x.split()])
lines.tail(1).eng.apply(lambda x: [ch for ch in x.split()])

they
got
into
the
train.


49999    [they, got, into, the, train.]
Name: eng, dtype: object

In [7]:
print(lines.tail(5))
lines.tail(5).eng.apply(lambda x: "".join(ch for ch in x if ch not in exclude))

                            eng                                 fr
49995  they gave us their word.  elles nous donnèrent leur parole.
49996  they go to work on foot.        ils vont au travail à pied.
49997  they got into the train.       ils montèrent dans le train.
49998  they got into the train.     elles montèrent dans le train.
49999  they got into the train.     ils sont montés dans le train.


49995    they gave us their word
49996    they go to work on foot
49997    they got into the train
49998    they got into the train
49999    they got into the train
Name: eng, dtype: object

In [8]:
lines.eng = lines.eng.apply(lambda x: "".join(ch for ch in x if ch not in exclude))
lines.fr = lines.fr.apply(lambda x: "".join(ch for ch in x if ch not in exclude))

In [9]:
print(lines.head(3))
print(lines.tail(3))

   eng         fr
0  run     cours 
1  run    courez 
2  wow  ça alors 
                           eng                             fr
49997  they got into the train    ils montèrent dans le train
49998  they got into the train  elles montèrent dans le train
49999  they got into the train  ils sont montés dans le train


In [10]:
lines.fr = lines.fr.apply(lambda x: 'start ' + x + ' end')
    # 디코더에 들어갈 프랑스어 문장에만 start 토큰과 end 토큰 추가
print(lines.head(3))
print(lines.tail(3))
print(lines.shape)

   eng                   fr
0  run     start cours  end
1  run    start courez  end
2  wow  start ça alors  end
                           eng                                       fr
49997  they got into the train    start ils montèrent dans le train end
49998  they got into the train  start elles montèrent dans le train end
49999  they got into the train  start ils sont montés dans le train end
(50000, 2)


In [11]:
from tensorflow.keras.preprocessing.text import Tokenizer

def create_tokenizer(lines):
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(lines)
    return tokenizer
    # 토크나이저를 문장에 적용해서 반환


import json
eng_tokenizer = create_tokenizer(lines['eng'])
eng_dict = json.loads(json.dumps(eng_tokenizer.word_counts))
    # tokenizer.word_counts에 저장된 단어빈도의 형태는 OrderedDict
    # json.dumps는 파이썬 객체를 json 문자열로 배열
    # json.loads는 다시 파이썬 객체로 역직렬화(그냥 파이썬 객체로 복원한다는 뜻)
df = pd.DataFrame([eng_dict.keys(), eng_dict.values()]).T
    # keys와 values가 각각 행으로 구성되므로, 두 열로 구성하기 위해 전치
df.columns = ['word', 'count']
df = df.sort_values(by = 'count', ascending = False)
    # 빈도기준 내림차순 정렬
df['cum_count'] = df['count'].cumsum()
df['cum_perc'] = df['cum_count']/df['cum_count'].max()
final_eng_words = df[df['cum_perc'] < 0.8]['word'].values

fr_tokenizer = create_tokenizer(lines['fr'])
fr_dict = json.loads(json.dumps(fr_tokenizer.word_counts))
    # 
df = pd.DataFrame([fr_dict.keys(), fr_dict.values()]).T
df.columns = ['word', 'count']
df = df.sort_values(by = 'count', ascending = False)
df['cum_count'] = df['count'].cumsum()
df['cum_perc'] = df['cum_count']/df['cum_count'].max()
final_fr_words = df[df['cum_perc'] < 0.8]['word'].values

print(len(final_eng_words), len(final_fr_words))

384 357


In [12]:
print(df.head(3))
print(df.tail(3))

     word  count cum_count  cum_perc
0   start  50000     50000  0.158862
2     end  50000    100000  0.317725
19     je   9886    109886  0.349135
           word count cum_count  cum_perc
9406    aperçus     1    314736  0.999994
4214   reprenez     1    314737  0.999997
14076    montés     1    314738       1.0


In [13]:
df[df['cum_perc']>0.8]['word'].values

array(['vue', 'dehors', 'sécurité', ..., 'aperçus', 'reprenez', 'montés'],
      dtype=object)

In [14]:
def filter_eng_words(x):
    t = []
    x = x.split()
    
    for i in range(len(x)):
        if x[i] in final_eng_words:
            t.append(x[i])
        else:
            t.append('unk')
    x3 = ""

    for i in range(len(t)):
        x3 = x3 + t[i] + ' '

    return x3

filter_eng_words('he is extremely good vue')

'he is unk good unk '

In [15]:
print(pd.DataFrame([[1,2,3],
            [4,5,6]]))
print(pd.DataFrame([[1,2,3],
            [4,5,6]]).apply(sum))

   0  1  2
0  1  2  3
1  4  5  6
0    5
1    7
2    9
dtype: int64


In [16]:
print(lines.tail(5).eng)
print(lines.tail(5).eng.apply(filter_eng_words))

49995    they gave us their word
49996    they go to work on foot
49997    they got into the train
49998    they got into the train
49999    they got into the train
Name: eng, dtype: object
49995     they gave us unk word 
49996    they go to work on unk 
49997     they got into the unk 
49998     they got into the unk 
49999     they got into the unk 
Name: eng, dtype: object


In [17]:
def filter_fr_words(x):
    t = []
    x = x.split()
    
    for i in range(len(x)):
        if x[i] in final_fr_words:
            t.append(x[i])
        else:
            t.append('unk')
    x3 = ""

    for i in range(len(t)):
        x3 = x3 + t[i] + ' '

    return x3

lines['eng'] = lines['eng'].apply(filter_eng_words)
lines['fr'] = lines['fr'].apply(filter_fr_words)
    # 자료형이 판다스 시리즈이므로 apply에 축 지정X

In [18]:
all_eng_words = set()
for eng in lines.eng:
    for word in eng.split():
        if word not in all_eng_words:
            all_eng_words.add(word)

all_french_words = set()
for fr in lines.fr:
    for word in fr.split():
        if word not in all_french_words:
            all_french_words.add(word)

input_words = sorted(list(all_eng_words))
target_words = sorted(list(all_french_words))
num_encoder_tokens = len(all_eng_words)
num_decoder_tokens = len(all_french_words)

print(num_encoder_tokens)
print(num_decoder_tokens)

385
358


In [19]:
print(set(all_french_words))
print(set(all_eng_words))

{'bonne', 'allé', 'a', 'perdu', 'peut', 'rester', 'ici', 'connais', 'jaimerais', 'vous', 'journée', 'être', 'crois', 'heureuse', 'besoin', 'pense', 'es', 'aucun', 'prends', 'mort', 'ma', 'nen', 'aller', 'soit', 'acheté', 'notre', 'nouveau', 'pourrait', 'mal', 'dun', 'lair', 'les', 'y', 'étions', 'ses', 'suis', 'elle', 'avait', 'estce', 'homme', 'si', 'vrai', 'moi', 'sur', 'fort', 'prêt', 'laissezmoi', 'tête', 'naime', 'parle', 'je', 'seule', 'reste', 'dis', 'sans', 'aime', 'jen', 'ceci', 'difficile', 'ton', 'déjà', 'peu', 'fatigué', 'ta', 'nes', 'jadore', 'il', 'vraiment', 'bon', 'me', 'temps', 'voici', 'son', 'aux', 'fois', 'confiance', 'ce', 'venir', 'astu', 'vais', 'on', 'tu', 'demain', 'ces', 'presque', 'malade', 'trop', 'beaucoup', 'tellement', 'verre', 'faire', 'toutes', 'pouvons', 'heures', 'vois', 'cette', 'prenez', 'veux', 'boulot', 'porte', 'comment', 'avezvous', 'end', 'du', 'viens', 'le', 'va', 'jétais', 'en', 'chercher', 'main', 'sais', 'ne', 'estil', 'devrais', 'prie', 'm

In [20]:
dict([(word, i) for i, word in enumerate(input_words)])

{'a': 0,
 'about': 1,
 'afraid': 2,
 'again': 3,
 'agree': 4,
 'all': 5,
 'almost': 6,
 'alone': 7,
 'already': 8,
 'always': 9,
 'am': 10,
 'an': 11,
 'and': 12,
 'angry': 13,
 'answer': 14,
 'any': 15,
 'anybody': 16,
 'anyone': 17,
 'anything': 18,
 'are': 19,
 'arent': 20,
 'as': 21,
 'ask': 22,
 'asked': 23,
 'at': 24,
 'away': 25,
 'back': 26,
 'bad': 27,
 'be': 28,
 'beautiful': 29,
 'bed': 30,
 'been': 31,
 'being': 32,
 'believe': 33,
 'best': 34,
 'better': 35,
 'bicycle': 36,
 'big': 37,
 'bit': 38,
 'book': 39,
 'books': 40,
 'both': 41,
 'bought': 42,
 'boy': 43,
 'bring': 44,
 'broke': 45,
 'busy': 46,
 'buy': 47,
 'by': 48,
 'call': 49,
 'called': 50,
 'came': 51,
 'can': 52,
 'cant': 53,
 'car': 54,
 'care': 55,
 'careful': 56,
 'cat': 57,
 'caught': 58,
 'close': 59,
 'coffee': 60,
 'cold': 61,
 'come': 62,
 'coming': 63,
 'could': 64,
 'crazy': 65,
 'day': 66,
 'dead': 67,
 'did': 68,
 'didnt': 69,
 'die': 70,
 'died': 71,
 'do': 72,
 'does': 73,
 'doesnt': 74,
 'dog'

In [21]:
input_token_index = dict(
    [(word, i+1) for i, word in enumerate(input_words)])

target_token_index = dict(
    [(word, i+1) for i, word in enumerate(target_words)])

print(input_token_index['unk'])
print(target_token_index['start'])
print(target_token_index['end'])
print(list(input_token_index.keys())[333])
print(list(target_token_index.keys())[283])
print(list(target_token_index.keys())[88])

334
284
89
unk
start
end


In [22]:
length_list = []
for l in lines.fr:
    length_list.append(len(l.split(' ')))
fr_max_length = np.max(length_list)

length_list = []
for l in lines.eng:
    length_list.append(len(l.split(' ')))
eng_max_length = np.max(length_list)

print(eng_max_length, fr_max_length)
    # RNN에서 타임스텝을 정의하기 위해 가장 긴 문장의 토큰 수를 구함

8 17


In [23]:
encoder_input_data = np.zeros((len(lines['eng']), eng_max_length), dtype='float32')
decoder_input_data = np.zeros((len(lines['fr']), fr_max_length), dtype='float32')
    # 개, 단
decoder_target_data = np.zeros((len(lines['fr']), fr_max_length, num_decoder_tokens+1), dtype='float32')

decoder_target_data.shape
    # 문장 개수, 시퀀스 길이, 총 단어 개수(why?: one-hot-encoding)
    # 개, 단, 차

(50000, 17, 359)

In [24]:
decoder_target_data[0][0].mean()

0.0

In [25]:
for i, (input_text, target_text) in enumerate(zip(lines['eng'], lines['fr'])):
    for t, word in enumerate(input_text.split()):
        encoder_input_data[i, t] = input_token_index[word]
    for t, word in enumerate(target_text.split()):
        decoder_input_data[i, t] = target_token_index[word]
        if t > 0:
            decoder_target_data[i, t-1, target_token_index[word]] = 1
            if t == len(target_text.split())-1:
                decoder_target_data[i, t:, 89] = 1

print(decoder_input_data.shape, encoder_input_data.shape, decoder_target_data.shape)

(50000, 17) (50000, 8) (50000, 17, 359)


In [26]:
encoder_input_data

array([[263.,   0.,   0., ...,   0.,   0.,   0.],
       [263.,   0.,   0., ...,   0.,   0.,   0.],
       [334.,   0.,   0., ...,   0.,   0.,   0.],
       ...,
       [310., 124., 161., ...,   0.,   0.,   0.],
       [310., 124., 161., ...,   0.,   0.,   0.],
       [310., 124., 161., ...,   0.,   0.,   0.]], dtype=float32)

In [27]:
decoder_input_data

array([[284., 320.,  89., ...,   0.,   0.,   0.],
       [284., 320.,  89., ...,   0.,   0.,   0.],
       [284., 351., 320., ...,   0.,   0.,   0.],
       ...,
       [284., 127., 320., ...,   0.,   0.,   0.],
       [284.,  86., 320., ...,   0.,   0.,   0.],
       [284., 127., 282., ...,   0.,   0.,   0.]], dtype=float32)

In [28]:
decoder_target_data[0][0][int(decoder_input_data[0][1])]

1.0

In [29]:
for i in range(decoder_input_data.shape[0]): # 개
    for j in range(decoder_input_data.shape[1]): # 단
        if (decoder_input_data[i][j]==0):
            decoder_input_data[i][j] = 89

# end 토큰 이후의 토큰들(실제로 0을 넣은 곳이 모두 pad 토큰임)까지 모두 end 토큰으로 통일

In [30]:
decoder_input_data[300]

array([284., 134., 285., 320.,  89.,  89.,  89.,  89.,  89.,  89.,  89.,
        89.,  89.,  89.,  89.,  89.,  89.], dtype=float32)

#### 여기서 왜 89를 넣는건가?
* target_token_index['end'] == 89
* decoder_input_data는 개, 단 행렬(원핫인코딩의 인덱스를 사용하므로 차는 존재하지 않음)
* decoder_input_data의 처음 인덱스 기준 i번째 데이터는 길이 17짜리 벡터이고, word1,...,wordn,<end>,0,...,0 형태
* 따라서 end 토큰 이후에 나오는 패딩 토큰들을 모두 <end> 처리하기 위한 목적임

### 13.2 seq2seq의 인코더만 사용해서 학습시키기

In [31]:
fr_max_length.dtype

dtype('int32')

In [32]:
encoder_input_data.shape

(50000, 8)

In [33]:
import tensorflow
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, LSTM, Dense, Embedding, Bidirectional, RepeatVector, Dropout

model1 = Sequential()
model1.add(Input(shape=(8,)))
model1.add(Embedding(input_dim=len(input_words)+1, output_dim=128))
    # 패딩토큰을 고려하여 input_dim은 단어 수보다 하나 많이
model1.add(Bidirectional(LSTM(256)))
model1.add(RepeatVector(int(fr_max_length)))
model1.add(LSTM(256, return_sequences=True))
model1.add(Dense(len(target_token_index)+1, activation='softmax'))

In [34]:
model1.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, 8, 128)         │        49,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ (None, 512)            │       788,480 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ repeat_vector (RepeatVector)    │ (None, 17, 512)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 17, 256)        │       787,456 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 17, 359)        │        92,263 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,717,607 (6.55 MB)

 Trainable params: 1,717,607 (6.55 MB)

 Non-trainable params: 0 (0.00 B)

In [38]:
model1.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])

history = model1.fit(encoder_input_data,
                     decoder_target_data,
                     batch_size=128,
                     epochs=5,
                     validation_split=0.05)

Epoch 1/5
372/372 ━━━━━━━━━━━━━━━━━━━━ 28s 69ms/step - acc: 0.7684 - loss: 1.5121 - val_acc: 0.7610 - val_loss: 1.2820
Epoch 2/5
372/372 ━━━━━━━━━━━━━━━━━━━━ 29s 77ms/step - acc: 0.8124 - loss: 0.9580 - val_acc: 0.7732 - val_loss: 1.1524
Epoch 3/5
372/372 ━━━━━━━━━━━━━━━━━━━━ 33s 90ms/step - acc: 0.8330 - loss: 0.8119 - val_acc: 0.7696 - val_loss: 1.1038
Epoch 4/5
372/372 ━━━━━━━━━━━━━━━━━━━━ 35s 94ms/step - acc: 0.8445 - loss: 0.7147 - val_acc: 0.7949 - val_loss: 0.9502
Epoch 5/5
372/372 ━━━━━━━━━━━━━━━━━━━━ 28s 74ms/step - acc: 0.8524 - loss: 0.6432 - val_acc: 0.7985 - val_loss: 0.9038


* validation accuracy가 약 80퍼로 잘나온듯하지만 이는 end(+pad) 토큰까지 포함된 결과

In [ ]:
decoder_target_data.shape # (개, 단, 차) 여기서 차는 원-핫 인코딩 벡터의 차원

(50000, 17, 359)

In [39]:
count = 0
correct_count = 0
pred = model1.predict(encoder_input_data[47500:])

for i in range(2500):
    t = np.argmax(pred[i], axis=-1)
    act = np.argmax(decoder_target_data[47500+i], axis=-1)
    correct_count += np.sum((act==t) & (act!=89))
    count += np.sum(act!=89)

print(correct_count, count)
print(correct_count/count)

79/79 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step
5117 13426
0.381126173096976


validation accuracy가 end(+pad) 토큰을 빼고 나니 40퍼도 되지않는 모습

In [40]:
print(pred.shape, pred[1].shape, np.argmax(pred[i], axis=-1).shape)

(2500, 17, 359) (17, 359) (17,)


In [ ]:
print(decoder_target_data[1])
print(decoder_target_data[1].shape)

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [45]:
encoder_input_data[-2400]

array([154.,  98., 184., 334.,   1., 334.,   0.,   0.], dtype=float32)

In [42]:
encoder_input_data[-2400].reshape(1, encoder_input_data.shape[1])

array([[154.,  98., 184., 334.,   1., 334.,   0.,   0.]], dtype=float32)

In [43]:
model1.predict(encoder_input_data[-2400].reshape(1, encoder_input_data.shape[1])).shape

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step


(1, 17, 359)

In [46]:
print(decoder_input_data.shape[1])
print(num_decoder_tokens+1)

17
359


In [47]:
model1.predict(encoder_input_data[-2400].reshape(1, encoder_input_data.shape[1])).reshape(decoder_input_data.shape[1], num_decoder_tokens+1).shape

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step


(17, 359)

In [50]:
t = model1.predict(encoder_input_data[-2400].reshape(1, encoder_input_data.shape[1])).reshape(decoder_input_data.shape[1], num_decoder_tokens+1)
t2 = np.argmax(t, axis=-1)

for i in range(len(t2)):
    if int(t2[i])!=0:
        print(list(target_token_index.keys())[int(t2[i]-1)])
        if int(t2[i])==89:
            break

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
je
me
unk
unk
unk
unk
end


### 디코더까지 추가해서 seq2seq 훈련시키기

In [51]:
Input(shape=(int(eng_max_length),))

<KerasTensor shape=(None, 8), dtype=float32, sparse=False, name=keras_tensor_16>

In [52]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Embedding, Bidirectional, RepeatVector, Dropout

# 인코더 입력층 정의
encoder_inputs = Input(shape=(int(eng_max_length),))  # 입력: 영어 시퀀스 최대 길이만큼의 정수 벡터
#   입력 형태는 (batch_size, eng_max_length)
#   batch_size는 들어오는 데이터에 따라 자동으로 설정됨

# 인코더 임베딩 층
en_x = Embedding(input_dim=num_encoder_tokens+1, output_dim=128)(encoder_inputs)  
#   num_encoder_tokens+1: 단어 집합 크기(= 데이터 내의 총 단어 수) (+1은 패딩토큰 포함)
#   why? Word2Vec 생각해보면 원-핫 벡터가 입력층에 들어가므로, 노드 수는 원-핫 벡터길이+1
#   128: 임베딩 벡터 차원

# 인코더 LSTM 정의
encoder = LSTM(units=256, return_state=True)  
#   units=256: LSTM 출력 차원 (= hidden state 벡터의 길이)
#   return_state=True: 마지막 은닉 상태와 셀 상태를 반환

# 인코더 LSTM 호출
encoder_outputs, state_h, state_c = encoder(en_x)
#   encoder_outputs: LSTM의 출력 (사용되지 않음)
#   state_h: 인코더의 마지막 은닉 상태
#   state_c: 인코더의 마지막 셀 상태

# 인코더 상태를 리스트로 저장
encoder_states = [state_h, state_c]  
#   디코더 LSTM의 초기 상태로 전달하기 위해 리스트로 저장

# 디코더 입력층 정의
decoder_inputs = Input(shape=(int(fr_max_length),))  
#   입력: 프랑스어 시퀀스 최대 길이만큼의 정수 벡터
#   입력 형태는 (batch_size, fr_max_length)

# 디코더 임베딩 층
de_x = Embedding(num_decoder_tokens+1, 128)(decoder_inputs)  
#   num_decoder_tokens+1: 단어 집합 크기 (+1은 패딩 포함)
#   128: 임베딩 벡터 차원

# 디코더 LSTM 정의
decoder_lstm = LSTM(units=256, return_sequences=True, return_state=True)  
#   units=256: LSTM 출력 차원
#   return_sequences=True: 모든 타임스텝의 출력을 반환
#   return_state=True: 마지막 은닉 상태와 셀 상태를 반환

# 디코더 LSTM 호출
decoder_outputs, _, _ = decoder_lstm(de_x, initial_state=encoder_states)  
#   decoder_outputs: 모든 타임스텝의 출력
#   _: 마지막 은닉 상태 (사용하지 않음)
#   _: 마지막 셀 상태 (사용하지 않음)

# 중간 Dense 층 추가 (비선형 변환)
decoder_outputs = Dense(2000, activation='tanh')(decoder_outputs)
#   2000: 은닉 노드 개수 
#   256 => 2000
#   tanh: 활성화 함수

# 최종 Dense 층 추가 (소프트맥스를 사용해 클래스 확률 분포 계산)
decoder_dense = Dense(num_decoder_tokens+1, activation='softmax')(decoder_outputs)  
#   num_decoder_tokens+1: 프랑스어 단어 집합 크기
#   2000 => 각 단어 수
#   softmax: 클래스(출력으로 나와야하는 단어) 확률 분포 계산

# 모델 정의 (인코더와 디코더 연결)
model2 = Model([encoder_inputs, decoder_inputs], decoder_dense)  
#   입력: 영어 시퀀스 (encoder_inputs)와 프랑스어 시퀀스 (decoder_inputs)
#   출력: 디코더 출력 (decoder_dense)
#   Model 함수를 통해 [encoder_inputs, decoder_inputs]로 decoder_dense를 구하기까지의
#   연산 경로를 알아서 추적하여 모델 구성해줌

# 모델 컴파일
model2.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])  
#   optimizer='adam': Adam 최적화 알고리즘
#   loss='categorical_crossentropy': 다중 클래스 분류 손실 함수
#   metrics=['acc']: 정확도(accuracy)를 메트릭으로 사용

# 모델 구조 출력
model2.summary()  
#   모델의 레이어 구성 및 파라미터 개수를 요약 출력


Model: "functional_5"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_2       │ (None, 8)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_3       │ (None, 17)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_1         │ (None, 8, 128)    │     49,408 │ input_layer_2[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_2         │ (None, 17, 128)   │     45,952 │ input_layer_3[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_2 (LSTM)       │ [(None, 256),     │    394,240 │ embedding_1[0][0] │
│                     │ (None, 256),      │            │                   │
│                     │ (None, 256)]      │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_3 (LSTM)       │ [(None, 17, 256), │    394,240 │ embedding_2[0][0… │
│                     │ (None, 256),      │            │ lstm_2[0][1],     │
│                     │ (None, 256)]      │            │ lstm_2[0][2]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 17, 2000)  │    514,000 │ lstm_3[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 17, 359)   │    718,359 │ dense_1[0][0]     │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 2,116,199 (8.07 MB)

 Trainable params: 2,116,199 (8.07 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
history2 = model2.fit([encoder_input_data, decoder_input_data],
                      decoder_target_data,
                      batch_size=128,
                      epochs=5,
                      validation_split=0.05)

Epoch 1/5


c:\Users\jongcheol\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\models\functional.py:225: UserWarning: The structure of `inputs` doesn't match the expected structure: ['keras_tensor_69', 'keras_tensor_74']. Received: the structure of inputs=('*', '*')
  warnings.warn(


372/372 ━━━━━━━━━━━━━━━━━━━━ 41s 103ms/step - acc: 0.7876 - loss: 1.3215 - val_acc: 0.8010 - val_loss: 0.9803
Epoch 2/5
372/372 ━━━━━━━━━━━━━━━━━━━━ 40s 106ms/step - acc: 0.8591 - loss: 0.6500 - val_acc: 0.8228 - val_loss: 0.7791
Epoch 3/5
372/372 ━━━━━━━━━━━━━━━━━━━━ 40s 106ms/step - acc: 0.8783 - loss: 0.5068 - val_acc: 0.8339 - val_loss: 0.6807
Epoch 4/5
372/372 ━━━━━━━━━━━━━━━━━━━━ 39s 104ms/step - acc: 0.8911 - loss: 0.4250 - val_acc: 0.8470 - val_loss: 0.6049
Epoch 5/5
372/372 ━━━━━━━━━━━━━━━━━━━━ 40s 106ms/step - acc: 0.9009 - loss: 0.3746 - val_acc: 0.8530 - val_loss: 0.5705


In [ ]:
np.argmax(decoder_target_data, axis=2).shape

(50000, 17)

In [ ]:
act = np.argmax(decoder_target_data, axis=2)
#   argmax를 통해 원-핫 벡터에서 1인 위치 즉, 정답 클래스의 위치를 구함
#   그럼 예측확률에 argmax를 취해 예측확률이 가장 높은 위치와 비교할 수 있기에
count=0
correct_count=0

pred = model2.predict([encoder_input_data[47500:], decoder_input_data[47500:]])

for i in range(2500):
    t = np.argmax(pred[i], axis=-1)
    correct_count += np.sum((act[47500+i]==t)&(act[47500+i]!=89))
    count += np.sum(decoder_input_data[47500+i]!=89)

print(correct_count, count)
print(correct_count/count)

79/79 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step
7368 15926
0.4626397086525179


#### Teacher forcing 없는 예측과정

In [ ]:
decoder_input_data_pred = np.zeros((len(lines['fr']), fr_max_length),
                                   dtype='float32') # (50000, 17)

final_pred = []
for i in range(2500):
    word = 284
    #   start 토큰의 인덱스
    for j in range(17):
        decoder_input_data_pred[(47500+i), j] = word
        pred = model2.predict([encoder_input_data[(47500+i)].reshape(1, 8),
                               decoder_input_data_pred[47500+i].reshape(1, 17)])
        #   print(pred.shape): (1, 17, 359)
        t = np.argmax(pred[0][j])
        word = t
        if word==89:
            break
    final_pred.append(list(decoder_input_data_pred[47500+i]))

final_pred = np.array(final_pred)
print(final_pred.shape)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━

In [ ]:
count = 0
correct_count = 0

for i in range(2500):
    correct_count += np.sum((decoder_input_data[47500+i]==final_pred[i])&(decoder_input_data[47500+i]!=89))
    count += np.sum(decoder_input_data[47500+i]!=89)

print(correct_count/count)

0.4870651764410398


### 13.4 Attention을 이용한 머신번역

In [31]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Embedding, Dropout, Activation, concatenate

In [32]:
encoder_inputs = Input(shape=(int(eng_max_length),)) # (8,)
en_x = Embedding(num_encoder_tokens+1, 128)(encoder_inputs)
en_x = Dropout(0.1)(en_x)
encoder_outputs, state_h, state_c = LSTM(256, return_sequences=True, return_state=True)(en_x)
encoder_states = [state_h, state_c]

decoder_inputs = Input(shape=(int(fr_max_length),)) # (17,)
de_x = Embedding(num_decoder_tokens+1, 128)(decoder_inputs)
decoder = Dropout(0.1)(de_x)
decoder = LSTM(256, return_sequences=True)(decoder, initial_state=encoder_states)
print('decoder', decoder)

decoder <KerasTensor shape=(None, 17, 256), dtype=float32, sparse=False, name=keras_tensor_9>


In [33]:
print(encoder_outputs.shape)
print(decoder.shape)

t = Dense(5000, activation='tanh')(encoder_outputs) # 들:256 나:5000
t1 = Dense(5000, activation='tanh')(decoder) # 동일

print(t)
print(t1)

(None, 8, 256)
(None, 17, 256)
<KerasTensor shape=(None, 8, 5000), dtype=float32, sparse=False, name=keras_tensor_10>
<KerasTensor shape=(None, 17, 5000), dtype=float32, sparse=False, name=keras_tensor_11>


In [34]:
from tensorflow.keras.layers import dot
attention = dot([t1, t], axes=[2,2]) # 2번째(실제로는 세번째임)축 즉, 256 벡터끼리 내적하는 층
attention # attention weight matrix들 (실제로는 아직 softmax를 적용안해서 가중치는 아님) 생성

<KerasTensor shape=(None, 17, 8), dtype=float32, sparse=False, name=keras_tensor_12>

In [35]:
attention = Dense(eng_max_length, activation='tanh')(attention)
#   이걸 왜 하지??

attention = Activation('softmax')(attention) # 이제 가중치값으로 행방향 정규화
print(attention.shape)

(None, 17, 8)


In [36]:
print(encoder_outputs.shape)

(None, 8, 256)


In [37]:
context = dot([attention, encoder_outputs], axes=[2,1]) # value vector들을 가중합
print('attention', attention)
print('context', context)

decoder_combined_context = concatenate([context, decoder])
print('decoder_combined_context', decoder_combined_context)
print(decoder_combined_context.shape)

attention <KerasTensor shape=(None, 17, 8), dtype=float32, sparse=False, name=keras_tensor_14>
context <KerasTensor shape=(None, 17, 256), dtype=float32, sparse=False, name=keras_tensor_15>
decoder_combined_context <KerasTensor shape=(None, 17, 512), dtype=float32, sparse=False, name=keras_tensor_16>
(None, 17, 512)


In [38]:
decoder_combined_context = Dense(2000, activation='tanh')(decoder_combined_context)
output = Dense(num_decoder_tokens+1, activation="softmax")(decoder_combined_context)
print('output', output)

output <KerasTensor shape=(None, 17, 359), dtype=float32, sparse=False, name=keras_tensor_18>


In [39]:
print(encoder_inputs.shape)
print(decoder_inputs.shape)
print(num_encoder_tokens+1)
print(num_decoder_tokens+1)

(None, 8)
(None, 17)
386
359


In [40]:
model3 = Model(inputs=[encoder_inputs, decoder_inputs], outputs=[output])
model3.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model3.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 8)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_1       │ (None, 17)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding           │ (None, 8, 128)    │     49,408 │ input_layer[0][0] │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_1         │ (None, 17, 128)   │     45,952 │ input_layer_1[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout (Dropout)   │ (None, 8, 128)    │          0 │ embedding[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_1 (Dropout) │ (None, 17, 128)   │          0 │ embedding_1[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm (LSTM)         │ [(None, 8, 256),  │    394,240 │ dropout[0][0]     │
│                     │ (None, 256),      │            │                   │
│                     │ (None, 256)]      │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_1 (LSTM)       │ (None, 17, 256)   │    394,240 │ dropout_1[0][0],  │
│                     │                   │            │ lstm[0][1],       │
│                     │                   │            │ lstm[0][2]        │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 17, 5000)  │  1,285,000 │ lstm_1[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 8, 5000)   │  1,285,000 │ lstm[0][0]        │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dot (Dot)           │ (None, 17, 8)     │          0 │ dense_1[0][0],    │
│                     │                   │            │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 17, 8)     │         72 │ dot[0][0]         │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation          │ (None, 17, 8)     │          0 │ dense_2[0][0]     │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dot_1 (Dot)         │ (None, 17, 256)   │          0 │ activation[0][0], │
│                     │                   │            │ lstm[0][0]        │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 17, 512)   │          0 │ dot_1[0][0],      │
│ (Concatenate)       │                   │            │ lstm_1[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_3 (Dense)     │ (None, 17, 2000)  │  1,026,000 │ concatenate[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_4 (Dense)     │ (None, 17, 359)   │    718,359 │ dense_3[0][0]     │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 5,198,271 (19.83 MB)

 Trainable params: 5,198,271 (19.83 MB)

 Non-trainable params: 0 (0.00 B)

### 각 레이어의 파라미터 수 계산식
* Embedding 레이어  
한 문장에 대한 임베딩 시 각 문장을 원-핫으로 변환 + 원-핫 벡터에 linear network 연결  
    - (num_encoder_tokens+1)x(128) = 386x128 = 49,408  
    - (num_decoder_tokens+1)x(128) = 359x128 = 45,952
* LSTM 레이어
    - (입력벡터길이)x(LSTM 셀의 수)x(게이트 수) + (은닉상태 크기)x(")x(") + (모든 LSTM 셀의 총 편향 수)x(")x(")
    - = 128x256x4 + 256x256x4 + 256x4 = 394,240
* Dense 레이어
    - 256x5000 + 5000 = 1,285,000
* Dense_3
    - 왜 있는지 모르겠으나, 굳이 의미부여 하자면 내적 결과에 FC 레이어를 이어붙여서 그 가중치까지도 학습하겠다?
    - 8x8 + 8 = 72
* Dense_4,5
    - concat된 벡터를 FC레이어에 통과시키는 부분
    - 512x2000 = 1,026,000
    - 2000x359 + 359 = 718,359

In [42]:
history3 = model3.fit([encoder_input_data, decoder_input_data],
                      decoder_target_data,
                      batch_size=32,
                      epochs=5,
                      validation_split=0.05)

Epoch 1/5


c:\Users\jongcheol\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\models\functional.py:225: UserWarning: The structure of `inputs` doesn't match the expected structure: ['keras_tensor', 'keras_tensor_6']. Received: the structure of inputs=('*', '*')
  warnings.warn(


1485/1485 ━━━━━━━━━━━━━━━━━━━━ 133s 88ms/step - accuracy: 0.8247 - loss: 0.9633 - val_accuracy: 0.8308 - val_loss: 0.6940
Epoch 2/5
1485/1485 ━━━━━━━━━━━━━━━━━━━━ 126s 85ms/step - accuracy: 0.8890 - loss: 0.4414 - val_accuracy: 0.8522 - val_loss: 0.5814
Epoch 3/5
1485/1485 ━━━━━━━━━━━━━━━━━━━━ 127s 85ms/step - accuracy: 0.9018 - loss: 0.3681 - val_accuracy: 0.8627 - val_loss: 0.5366
Epoch 4/5
1485/1485 ━━━━━━━━━━━━━━━━━━━━ 124s 84ms/step - accuracy: 0.9096 - loss: 0.3301 - val_accuracy: 0.8657 - val_loss: 0.5107
Epoch 5/5
1485/1485 ━━━━━━━━━━━━━━━━━━━━ 121s 82ms/step - accuracy: 0.9145 - loss: 0.3051 - val_accuracy: 0.8668 - val_loss: 0.5113


In [43]:
decoder_input_data_pred = np.zeros((len(lines.fr), fr_max_length),
                                   dtype='float32')

final_pred_att = []
for i in range(2500):
    word = 284
    for i in range(17):
        decoder_input_data_pred[(47500+i), j] = word
        pred = model3.predict([encoder_input_data[(47500+i)].reshape(1,8),
                               decoder_input_data_pred[47500+i].reshape(1,17)])
        t = np.argmax(pred[0][j])
        word = t
        if word==89:
            break
    final_pred_att.append(list(decoder_input_data_pred[47500+i]))

final_pred2_att = np.array(final_pred_att)
count = 0
correct_count = 0

for i in range(2500):
    correct_count += np.sum((decoder_input_data[47500+i] == final_pred2_att[i]) &
                            (decoder_input_data[47500+i] != 89))
    count += np.sum(decoder_input_data[47500+i] != 89)

print("Validation Accuracy: ", correct_count/count)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 214ms/step

c:\Users\jongcheol\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\models\functional.py:225: UserWarning: The structure of `inputs` doesn't match the expected structure: ['keras_tensor', 'keras_tensor_6']. Received: the structure of inputs=('*', '*')
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 214ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━

KeyboardInterrupt: 

In [45]:
k = -2400
t = model3.predict([encoder_input_data[k].reshape(1,encoder_input_data.shape[1]),
                    decoder_input_data[k].reshape(1,decoder_input_data.shape[1])]).reshape(
                        decoder_input_data.shape[1], num_decoder_tokens+1
                    )
for i in range(len(encoder_input_data[k])):
    if int(encoder_input_data[k][i])!=0:
        print(list(input_token_index.keys())[int(encoder_input_data[k][i]-1)])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
i
feel
like
unk
a
unk


In [ ]:
t2 = np.argmax(t, axis=1)
dd
for i in range(len(t2)):
    if int(t2[i])!=0:
        print(list(target_token_index.keys())[int(t2[i]-1)])
        if int(t2[i]==89):
            break

je
unk
unk
unk
unk
end


In [48]:
t2 = decoder_input_data[k]

for i in range(len(t2)):
    if int(t2[i])!=0:
        print(list(target_token_index.keys())[int(t2[i]-1)])
        if int(t2[i])==89:
            break

start
jai
unk
dêtre
un
unk
unk
end
